In [2]:
pip install gradio

  Using cached gradio-5.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.4.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.12-py3-none-any.whl.metadata (1.9 kB)
  Using cached ruff-0.7.1-py3-none-macosx_10_12_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached websockets-12.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.6 kB)
Using cached gradio-5.4.0-py3-none-any.whl (56.7 MB)
Using cached gradio_client-1.4.2-py3-none-any.whl (319 kB)
Using cached python_multipart-0.0.12-py3-none-any.whl (23 kB)
Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
Usi

In [11]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr

In [12]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    X = sms_text_df['text_message']
    # Set the target variable to the "label" column.
    y = sms_text_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LinearSVC())
    ])

    # Fit the model to the transformed training data and return model.
    text_clf.fit(X_train, y_train)
    return text_clf

In [13]:
# Load the dataset into a DataFrame
sms_data = "./Resources/SMSSpamCollection.csv"
sms_df = pd.read_csv(sms_data)
sms_df.head()

,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [14]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_df)
text_clf

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [22]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    
    # Using a conditional if the prediction is "ham" return the message:
    
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    

    prediction = text_clf.predict([text])[0]

    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'
txt_message = "Win 1000 dollars now! Text WIN to 12345."
result = sms_prediction(txt_message)
print(result)

The text message: "Win 1000 dollars now! Text WIN to 12345.", is spam.


In [23]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.  
# Provide labels for each textbox. 
def sms_prediction(text):
    prediction = text_clf.predict([text])[0]
    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'
    
sms_app = gr.Interface(
    fn=sms_prediction,  # the function to call
    inputs=gr.Textbox(label="Enter SMS Text"),  # input textbox
    outputs=gr.Textbox(label="Prediction Result"),  # output textbox
    title="SMS Spam Detection",  # Title of the app
    description="Type an SMS text message to see if it's classified as spam or not spam."  # Description of what the app does
)

# Launch the app.
sms_app.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.

1. Not spam
2. Spam
3. Spam
4. Spam